# Operações Avançadas com GeoPandas

Se as operações básicas são como usar régua e compasso, as operações avançadas são como usar uma estação total ou um GPS RTK - mais complexas, mas muito mais poderosas! Vamos aprender a fazer análises espaciais sofisticadas usando código.

## Objetivos
- Realizar operações espaciais complexas (como overlay avançado)
- Trabalhar com relacionamentos topológicos (como análise de redes)
- Executar análises espaciais avançadas (como clustering)
- Criar visualizações complexas (como mapas de calor)

## 1. Configuração do Ambiente

Vamos preparar nosso laboratório digital de análise espacial:

In [ ]:
# Nossa estação de trabalho avançada:
import geopandas as gpd              # Nossa ferramenta SIG
import pandas as pd                  # Nossa planilha de cálculo
import matplotlib.pyplot as plt       # Nossa mesa de desenho
from shapely.geometry import Point, Polygon, LineString  # Nossas ferramentas CAD
from shapely.ops import unary_union  # Nossa ferramenta de dissolve
import numpy as np                   # Nossa calculadora científica

plt.rcParams['figure.figsize'] = (12, 8)

## 2. Operações Topológicas

### 2.1 Interseção, União e Diferença

Estas operações são como usar papel vegetal para sobrepor mapas:

In [ ]:
# Criando polígonos de exemplo
# Como desenhar duas áreas que se sobrepõem
poligono1 = Polygon([(0, 0), (0, 2), (2, 2), (2, 0)])
poligono2 = Polygon([(1, 1), (1, 3), (3, 3), (3, 1)])

# Criando GeoDataFrames
# Como preparar nossas camadas para análise
gdf1 = gpd.GeoDataFrame({'geometry': [poligono1], 'nome': ['A']})
gdf2 = gpd.GeoDataFrame({'geometry': [poligono2], 'nome': ['B']})

# Operações topológicas
# Como usar as ferramentas de geoprocessamento
intersecao = gpd.overlay(gdf1, gdf2, how='intersection')  # Como 'Intersect'
uniao = gpd.overlay(gdf1, gdf2, how='union')             # Como 'Union'
diferenca = gpd.overlay(gdf1, gdf2, how='difference')     # Como 'Erase'

# Visualização
# Como ver os resultados de cada operação
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# Mostrando a interseção
gdf1.plot(ax=ax1, alpha=0.5, color='red')
gdf2.plot(ax=ax1, alpha=0.5, color='blue')
intersecao.plot(ax=ax1, color='purple')
ax1.set_title('Interseção (área comum)')

# Mostrando a união
uniao.plot(ax=ax2, color='green')
ax2.set_title('União (todas as áreas)')

# Mostrando a diferença
diferenca.plot(ax=ax3, color='orange')
ax3.set_title('Diferença (A menos B)')

plt.tight_layout()
plt.show()

### 2.2 Análise de Vizinhança

In [ ]:
# Carregando dados dos estados
# Como abrir nossa camada de divisão política
estados = gpd.read_file('../data/raw/BR_UF_2022.shp')

# Encontrando vizinhos de São Paulo
# Como usar 'Select Adjacent Features'
sp = estados[estados['sigla'] == 'SP']
vizinhos = estados[estados.geometry.touches(sp.geometry.iloc[0])]

# Visualização
# Como destacar um estado e seus vizinhos
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
estados.plot(ax=ax, color='lightgray')
sp.plot(ax=ax, color='red', alpha=0.6)
vizinhos.plot(ax=ax, color='blue', alpha=0.4)

plt.title('São Paulo e seus Estados Vizinhos')
plt.show()

print("Estados que fazem fronteira com São Paulo:")
print(vizinhos['nome'].tolist())

## 3. Análise de Distância e Proximidade

In [ ]:
# Criando pontos de interesse (POIs)
# Como marcar locais importantes no mapa
pois = gpd.GeoDataFrame(
    {'nome': ['Shopping A', 'Shopping B', 'Shopping C', 'Shopping D'],
     'geometry': [Point(-46.65, -23.55), Point(-46.64, -23.54),
                 Point(-46.63, -23.53), Point(-46.62, -23.52)]},
    crs="EPSG:4326"
)

# Convertendo para UTM para cálculos em metros
# Como mudar para uma projeção que permite medir distâncias
pois_utm = pois.to_crs(epsg=32723)

# Matriz de distância
# Como calcular distâncias entre todos os pontos
matriz_dist = pd.DataFrame(
    index=pois_utm['nome'],
    columns=pois_utm['nome'],
    data=[[g1.distance(g2) for g2 in pois_utm.geometry] 
          for g1 in pois_utm.geometry])

print("Matriz de distâncias em metros:")
print(matriz_dist.round(2))

## 4. Análise de Densidade e Clusters

In [ ]:
# Criando pontos aleatórios
# Como simular uma distribuição de ocorrências
np.random.seed(42)
n_points = 100
points = gpd.GeoDataFrame(
    {'geometry': [Point(xy) for xy in np.random.rand(n_points, 2)],
     'valor': np.random.rand(n_points)}
)

# Criando grid para análise de densidade
# Como fazer um mapa de calor (heatmap)
from shapely.geometry import box

def create_grid(bounds, nx, ny):
    """Cria uma grade regular (como quadrículas de análise)"""
    xmin, ymin, xmax, ymax = bounds
    cell_width = (xmax - xmin) / nx
    cell_height = (ymax - ymin) / ny
    
    grid_cells = []
    for i in range(nx):
        for j in range(ny):
            x1 = xmin + i * cell_width
            y1 = ymin + j * cell_height
            x2 = x1 + cell_width
            y2 = y1 + cell_height
            grid_cells.append(box(x1, y1, x2, y2))
    
    return gpd.GeoDataFrame({'geometry': grid_cells})

grid = create_grid(points.total_bounds, 10, 10)

# Contando pontos em cada célula
# Como fazer análise de densidade por quadrícula
grid['count'] = grid.geometry.apply(lambda cell: sum(points.geometry.intersects(cell)))

# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

points.plot(ax=ax1, markersize=50, alpha=0.5)
ax1.set_title('Distribuição dos Pontos')

grid.plot(column='count', legend=True, cmap='YlOrRd', ax=ax2)
ax2.set_title('Análise de Densidade')

plt.tight_layout()
plt.show()

## 5. Análise de Redes

In [ ]:
# Criando uma rede simples de ruas
# Como digitalizar um sistema viário
ruas = gpd.GeoDataFrame(
    {'nome': ['Rua A', 'Rua B', 'Rua C', 'Rua D'],
     'geometry': [
         LineString([(0, 0), (1, 1)]),      # Como uma via principal
         LineString([(1, 1), (2, 1)]),      # Como uma continuação
         LineString([(1, 1), (1, 2)]),      # Como uma transversal
         LineString([(2, 1), (2, 2)])       # Como uma paralela
     ]}
)

# Encontrando intersecções
# Como identificar cruzamentos
intersecoes = []
for idx1, rua1 in ruas.iterrows():
    for idx2, rua2 in ruas.iterrows():
        if idx1 < idx2:  # Evita duplicatas
            if rua1.geometry.intersects(rua2.geometry):
                ponto = rua1.geometry.intersection(rua2.geometry)
                intersecoes.append({
                    'geometry': ponto,
                    'ruas': f"{rua1['nome']} x {rua2['nome']}"
                })

intersecoes_gdf = gpd.GeoDataFrame(intersecoes)

# Visualização
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ruas.plot(ax=ax, color='blue')
intersecoes_gdf.plot(ax=ax, color='red', markersize=100)
plt.title('Rede de Ruas e seus Cruzamentos')
plt.show()

## 6. Exercícios Práticos

Vamos praticar como um analista GIS avançado:

1. Análise de Áreas de Influência:
   - Carregue pontos de escolas
   - Crie buffers de 1km (como raio de atendimento)
   - Identifique áreas com sobreposição
   - Calcule população atendida

2. Análise de Conectividade:
   - Use dados do sistema viário
   - Identifique nós principais
   - Calcule métricas de rede
   - Visualize áreas mais conectadas

3. Análise de Clusters:
   - Use dados de ocorrências
   - Identifique hot spots
   - Crie mapas de calor
   - Sugira intervenções baseadas na análise